### Logistic differential equation

In [ ]:
def population_next_generation(growth_rate: float, population: float) -> float:
    r: float = growth_rate
    x: float = population
    return r * x * (1 - x)

def population_nth_generation(growth_rate: float, population_history: list, generations: int) -> list:
    r: float = growth_rate
    n: int = generations
    x: list = population_history
    x0: float = population_history[-1]

    x1 = population_next_generation(r, x0)
    x = x + [x1]
    n = n - 1
    
    if n > 0:
        return population_nth_generation(r, x, n)
    else:
        return x

### Helper functions

In [ ]:
import matplotlib.pyplot as plt

def plot_population_over_time(population_history: list):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1) 

    x = range(len(population_history))
    y = population_history
        
    ax.cla()
    ax.plot(x, y)

### Next generation population

In [ ]:
r = 2.5
x = 0.5

In [ ]:
x = population_next_generation(r, x)
x

### Calculate all generation's population

In [ ]:
r = 3.0
x = 0.5
n = 100

X = population_nth_generation(r, [x], n)
plot_population_over_time(X)